<a href="https://colab.research.google.com/github/FrancescoRosi01/Tesi/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import matplotlib.pyplot as plt
from pathlib import Path
import json
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [91]:
# clona la repo con i dataset
# i dataset sono in ./Tesi/datasets/
# il primo dataset è ./Tesi/datasets/ds
!git clone https://github.com/FrancescoRosi01/Tesi.git

fatal: destination path 'Tesi' already exists and is not an empty directory.


In [92]:
def load_and_process_image(path, label):
    """ restituisce il tensore 3d (l'immagine) e la label associata al path che prende in input"""
    img_raw = tf.io.read_file(path)
    img = tf.io.decode_png(img_raw, channels=3)
    img = tf.image.resize(img, (224,224))
    # normalizzare
    img = img / 255.0
    return img, label

def load_ds(ds_path : Path):
    """ restituisce i dataset di training, validation e test"""
    imgs_path = ds_path / 'images'
    labels_path = ds_path / 'labels' / 'labels.json'
    with open(labels_path, 'r') as f:
        labels = json.load(f)
        labels = np.array(list(labels.values()))
        labels = labels.astype(np.float32)
    regex = re.compile(r"img_(\d+)\.png")
    # imgs_path è una lista ordinata di path delle immagini.
    imgs_path = sorted(
        map(lambda x:str(x),imgs_path.glob('img_*.png')),
        key=lambda x: int(regex.search(x).group(1))
        )
    train_paths, test_paths, train_labels, test_labels = train_test_split(imgs_path, labels, test_size=0.2, random_state=1)
    train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.25, random_state=1)
    # train -> 60%, val -> 20%, test -> 20%
    train_ds = tf.data.Dataset.from_tensor_slices((train_paths,train_labels))
    val_ds = tf.data.Dataset.from_tensor_slices((val_paths,val_labels))
    test_ds = tf.data.Dataset.from_tensor_slices((test_paths,test_labels))
    BATCH_SZ = 32
    # load_and_process_image restituisce il tensore 3d (l'immagine) associata al path che prende in input
    train_ds = train_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)
    # Shuffle
    train_ds = train_ds.shuffle(buffer_size=1000)
    val_ds = val_ds.shuffle(buffer_size=1000)
    # Batch
    train_ds = train_ds.batch(BATCH_SZ)
    val_ds = val_ds.batch(BATCH_SZ)
    test_ds = test_ds.batch(BATCH_SZ)
    train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size= tf.data.AUTOTUNE)
    return train_ds,val_ds,test_ds

In [93]:
train,val,test = load_ds(Path("Tesi/datasets/ds"))

In [94]:
for images, labels in train.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    print(f"Sample label: {labels[0]}")

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)
Sample label: 0.24444444477558136


In [95]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(224,224,3)),

    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu',kernel_initializer="he_normal"),
    keras.layers.Dropout(0.5),
    # tanh come attivazione finale perchè resituisce valori tra -1 e 1
    keras.layers.Dense(1, activation='tanh')
    ])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae'],
)

In [96]:
# su CPU Colab il training per il primo ds dura circa 5 min
history = model.fit(
    train,
    epochs = 10,
    validation_data = val,
    callbacks=[keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)]
    )


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 0.3478 - mae: 0.4589 - val_loss: 0.0413 - val_mae: 0.1691
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - loss: 0.0479 - mae: 0.1739 - val_loss: 0.0129 - val_mae: 0.0891
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - loss: 0.0175 - mae: 0.1053 - val_loss: 0.0066 - val_mae: 0.0650
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 0.0168 - mae: 0.0953 - val_loss: 0.0052 - val_mae: 0.0613
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - loss: 0.0116 - mae: 0.0849 - val_loss: 0.0029 - val_mae: 0.0442
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 0.0107 - mae: 0.0784 - val_loss: 0.0023 - val_mae: 0.0394
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 0.0090 - mae: 0.0695 - val_loss: 0.0028 - val_mae: 0.0428
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - loss: 0.0124 - mae: 0.0857 - val_loss: 0.0032 - val_mae: 0.0468
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 0.0101 - mae: 

In [97]:
model.evaluate(test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 559ms/step - loss: 0.0029 - mae: 0.0430


[0.002505873329937458, 0.039590172469615936]

Si vede che il modello performa bene quando


1.   le immagini hanno sfondo nero e linea bianca
2.   l'offset è nel range -90, -90

il punto 1 è abbastanza comprensibile dato che il modello è stato addestrato solo su immagini con background nero e linea bianca.

